# 第19章 MCMC

## 19.0 背景

- 频率角度——优化问题  
$$\hat{\theta}=\arg\max_{\theta} \log P(X|\theta)\underset{iid}{=}\arg\max_{\theta} \sum_{i=1}^{N} \log P(x_{i}|\theta)$$
比如: 逻辑回归与最大熵模型、SVM、EM算法  


- 贝叶斯角度——积分问题  
后验: 
$$P(\theta|X) = \frac{P(X|\theta) P(\theta)}{P(X)} = \frac{P(X|\theta) P(\theta)}{\int_{\theta}P(X|\theta) P(\theta)}$$  
参数估计: $$\theta_{MAP} = \arg \max_{\theta}P(\theta|X)=\arg \max_{\theta}P(X|\theta)P(\theta)$$

贝叶斯推断:  
核心在于求解参数后验分布$P(\theta|X)$，或求目标函数在后验分布上的期望$\mathbb{E}_{\theta|X}[f(x)]$


$$
\text{Inference} = \left\{\begin{matrix}
精确推断 \\
近似推断 = \left\{\begin{matrix}
 确定性: \text{VI}\\
 随机: \text{MCMC}
\end{matrix}\right.
\end{matrix}\right.
$$

1. 蒙特卡罗法（Monte Carlo Method），也称统计模拟方法，<font color=blue>通过从概率模型的随机抽样进行近似数值计算的方法。</font>  


2. MCMC，是以马尔科夫链为概率模型的蒙特卡罗法。MCMC构建一个马尔科夫链，使其平稳分布就是要进行抽样的分布，首先基于该马尔科夫链进行随机游走，产生样本的序列，之后使用该平稳分布的样本进行近似数值计算。

## 19.1 蒙特卡罗法

### 19.1.0 前置知识

1. 采样的动机  
    - 采样本身就是常见的任务
    - 求和或求积分


2. 什么是好样本
    - 样本区域高概率区域
    - 样本之间相互独立
    
    
    
3. 采样是困难的
    - partation function is intractable
    - high dimension

### 19.1.1 随机抽样

1. 蒙特卡洛要解决的问题:  
假设概率分布$p(x)$已知，通过抽样获取概率分布的随机样本，并通过得到的随机样本对概率分布的特征进行分析。  


2. 一般的蒙特卡洛法有:  
直接抽样法、接受-拒绝抽样法、重要性抽样法

#### 1. 直接抽样法

<img src='19.1_直接抽样法.jpg' style='zoom:50%'/>

p.d.f = $p(x)$；c.d.f $F(x) = \int p(x)dx$  

1. 从U\[0,1\]抽取随机数$\xi$;  
2. 令$F(x)=\xi$;  
3. 解$F^{-1}(x)$得到样本$x$

直接抽样法的问题是并不是所有的$F(x)$都容易求得。

#### 2. 接受-拒绝抽样（accept-reject sampling method）

<img src='19.1_接受-拒绝抽样法.png' style='zoom:50%'/>

输入: 抽样目标的概率密度函数$p(x)$； 

输出: 概率分布的随机样本$x_{1}, x_{2}, \cdots, x_{n}$  

参数: $n$

（1）选择建议分布$q(x)$，使其对任意$x$满足$cq(x) \geq p(x)$，其中$c>0$；  

（2）按照建议分布$q(x)$随机抽样得到$x^{*}$，再按均匀分布$U[0,1]$抽样得到$u$； 

（3）如果$u \leq \frac{p(x^{*})}{cq(x^{*})}$，则将$x^{*}$作为抽样结果；否则放回，回到步骤(2)。  

（4）直到得到$n$个样本，结束。

接受-拒绝法优点: 容易实现  

缺点: <font color=blue>效率可能不高。如果$p(x)$的涵盖体积占$cq(x)$涵盖体积的比例很低，就会导致拒绝的比例很高，抽样效率很低。</font>

### 9.1.2 数学期望估计

假设随机变量$x \in \mathcal{X}$，其p.d.f为$p(x)$，$f(x)$为定义在$\mathcal{X}$上的函数，目标是求数学期望$E_{p(x)}[f(x)]$。  

蒙特卡洛法依概率$p(x)$独立地抽取$n$个样本$x_{1}, x_{2}, \cdots, x_{n}$，之后用函数的样本均值作为期望的近似:  

$$
E_{p(x)}[f(x)] \approx \frac{1}{n}\sum_{i=1}^{n}f(x_{i}) \tag{19.3}
$$

### 9.1.3 积分计算

假设有一个函数$h(x)$，目标是计算该函数的积分  

$$\int_{\mathcal{X}} h(x) \mathrm{d} x$$

可以将$h(x)$分解为一个函数$f(x)$与一个p.d.f$p(x)$的乘积形式，那么有  

$$
\int_{\mathcal{X}} h(x) \mathrm{d} x=\int_{\mathcal{X}} f(x) p(x) \mathrm{d} x=E_{p(x)}[f(x)] \tag{19.4}
$$

<font color=blue>任何一个函数的积分都可以表示为某一个函数的数学期望的形式。</font>而函数的数学期望可以用样本均值估计，于是有  

$$
\int_{\mathcal{X}} h(x) \mathrm{d} x=E_{p(x)}[f(x)] \approx \frac{1}{n} \sum_{i=1}^{n} f\left(x_{i}\right) \tag{19.5}
$$

一般的蒙特卡洛法中的抽样样本是独立的，而MCMC抽样样本不是独立的，样本序列形成马尔科夫链。

## 19.2 马尔科夫链

### 19.2.1 基本定义

#### 定义19.1（马尔科夫链）

考虑一个随机变量的序列$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，这里$X_{t}$表示$t$时刻的随机变量。每个随机变量$X_{t}(t=1,2,\cdots)$的<font color=blue>取值集合相同，称为状态空间，表示为$\mathcal{S}$</font>。随机变量可以是离散的，也可以是连续的。以上随机变量序列构成<font color=blue>随机过程（stochastic process）</font>。  


假设0时刻的随机变量$X_{0}$的概率分布$P(X_{0})=\pi_{0}$，称为<font color=blue>初始状态分布</font>。马尔可夫性:  

$$
P\left(X_{t} \mid X_{0}, X_{1}, \cdots, X_{t-1}\right)=P\left(X_{t} \mid X_{t-1}\right), \quad t=1,2, \cdots \tag{19.6}
$$

<font color=blue>具有马尔可夫性的随机序列$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$称为</font><font color=red>马尔科夫链</font>，或<font color=red>马尔科夫过程</font>。条件概率分布$P\left(X_{t} | X_{t-1}\right)$称为马尔科夫链的<font color=blue>转移概率分布</font>。转移概率分布决定了马尔科夫链的特性。

若转移概率分布$P\left(X_{t+s} | X_{t-1+s}\right)$与$t$无关，即

$$
P\left(X_{t+s} | X_{t-1+s}\right)=P\left(X_{t} | X_{t-1}\right), \quad t=1,2, \cdots ; \quad s=1,2, \cdots \tag{19.7}
$$ 

则称该马尔科夫链为<font color=blue>时间齐次的马尔科夫链（time homogenous Markov chain）</font>

### 19.2.2 离散状态马尔科夫链

#### 1. 转移概率矩阵和状态分布

- **转移概率矩阵**

若$(t-1)$时刻状态为$j$，在时刻$t$移动到状态$i$，将转移概率记作  

$$
p_{i j}=\left(\color{red}{X_{t}=i | X_{t-1}=j} \right), \quad i=1,2, \cdots ; \quad j=1,2, \cdots \\
\text{s.t.} \quad p_{\color{red}{i j}} \geq 0, \quad \sum_{i}p_{ij}=1 \tag{19.9}
$$

写成矩阵形式  

$$
\begin{align}
& \ \tiny{\sum=1} \ \tiny{\sum=1} \ \ \tiny{\sum=1} \ \ \cdots \\
P=&\left[\begin{array}{cccc}p_{11} & p_{12} & p_{13} & \cdots \\ p_{21} & p_{22} & p_{23} & \cdots \\ p_{31} & p_{32} & p_{33} & \cdots \\ \cdots & \cdots & \cdots & \cdots\end{array}\right] \tag{19.10}
\end{align}
$$

- **状态分布**  

马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$在时刻$t$的概率分布，称为时刻$t$的<font color=blue>状态分布</font>，记作  

$$
\pi(t)=\left[\begin{array}{c}\pi_{1}(t) \\ \pi_{2}(t) \\ \vdots\end{array}\right] \tag{19.11}
$$

其中$\pi_{i}(t)$表示时刻$t$状态为$i$的概率  

$$\pi_{i}(t)=P\left(X_{t}=i\right), \quad i=1,2, \cdots$$

初始状态可表示为  

$$
\pi(0)=\left[\begin{array}{c}\pi_{1}(0) \\ \pi_{2}(0) \\ \vdots\end{array}\right] \tag{19.12}
$$

<font color=blue>通常，初始分布$\pi(0)$的向量只有一个分类是1，其余分量都是0，表示马尔科夫链从一个具体状态开始。</font>

<img src='19.2_马尔科夫链例.png' style='zoom:50%'/>

马尔科夫链$X$在时刻$t$的状态分布，可以由时刻$(t-1)$的状态分布以及转移概率分布决定  

$$
\pi(t)=P \pi(t-1) \tag{19.13}
$$

递推得到  

$$
\pi(t)=P^{t} \pi(0) \tag{19.14}
$$

<font color=blue>式（19.14）说明，马尔科夫链的状态分布由初始分布和转移概率分布决定。</font>

#### 2. <font color=red>平稳分布</font>

**定义19.2（平稳分布）**  

设有马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，其状态空间为$\mathcal{S}$，转移概率矩阵为$P=[p_{ij}]$，如果存在状态空间$\mathcal{S}$上的一个分布  

$$\pi=\left[\begin{array}{c}\pi_{1} \\ \pi_{2} \\ \vdots\end{array}\right]$$  

使得  

$$
\color{red}{\pi = P \pi} \tag{19.15}
$$  

则<font color=red>称$\pi$</font>为马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$的<font color=red>平稳分布</font>。

<font color=blue>直观上，如果马尔科夫链的平稳分布存在，那么以该平稳分布作为初始分布，面向未来进行随机状态转移，之后任何一个时刻的状态分布都是该平稳分布。</font>

**引理19.1**  

给定一个马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，其状态空间为$\mathcal{S}$，转移概率矩阵为$P=[p_{ij}]$，则分布$\pi = (\pi_{1}, \pi_{2}, \cdots)^{\mathrm{T}}$为$X$的平稳分布的充要条件是$\pi = (\pi_{1}, \pi_{2}, \cdots)^{\mathrm{T}}$是下列方程组的解:  

$$
\begin{eqnarray}
&& x_{i}=\sum_{j} p_{i j} x_{j}, \quad i=1,2, \cdots \tag{19.16} \\
&&  x_{i} \geq 0, \quad i=1,2, \cdots \tag{19.17} \\
&& \sum_{i}x_{i}=1 \tag{19.18}
\end{eqnarray}
$$

<font color=blue>马尔科夫链可能存在唯一平稳分布，无穷多个平稳分布，或不存在平稳分布。</font><font color=red>可以看做是求$P$的特征向量: 取特征值为1对应的特征向量，并且对特征向量元素进行等比例缩放，使其元素和等于1.</font>

### 19.2.3 连续状态马尔科夫链

连续状态马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，随机变量$X_{t}$定义在连续状态空间$\mathcal{S}$，转移概率分布由概率转移核或<font color=blue>转移核（transition kernel）</font>表示。  

设$\mathcal{S}$是连续状态空间，对任意的$x \in \mathcal{S},A \subset{\mathcal{S}}$，转移核$P(x,A)$定义为  

$$
\begin{eqnarray}
P(x,A) &=& \int_{A}p(x,y)\text{d}y \\ 
&=& P(X_{t}=A|X_{t-1}=x)\tag{19.19}
\end{eqnarray}
$$

其中$p(x,\bullet)$是概率密度函数，满足$p(x,\bullet) \geq 0,\ P(x,\mathcal{S}) = \int_{\mathcal{S}}p(x,y)\text{d}y=1$

若马尔科夫链的状态空间$\mathcal{S}$上的概率分布$\pi(x)$满足条件  

$$
\color{blue}{\pi(y) = \int p(x,y)\pi(x)\text{d}x, \quad \forall y \in \mathcal{S}} \tag{19.21}
$$

则称分布$\pi(x)$为马尔科夫链的<font color=red>平稳分布</font>。等价地，  

$$
\pi(A) = \int p(x,A)\pi(x)\text{d}x, \quad \forall A \subset \mathcal{S} \tag{19.22}
$$

### 19.2.4 马尔科夫链的性质

#### 1. 不可约（irreducible）

设有马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，其状态空间为$\mathcal{S}$，对任意状态$i,j \in \mathcal{S}$，如果存在一个时刻$t(t > 0)$满足  

$$
P(X_{t}=i|X_{\color{red}{0}}=j)>0 \tag{19.24}
$$

则称此马尔科夫链$X$是不可约的（irreducible），否则称为马尔科夫链是可约的（reducible）。

直观上，<font color=blue>一个不可约的马尔科夫链，从任意状态出发，经过充分长时间后，可以达到任意状态。</font>  

<img src='19.2_马尔科夫链不可约.png' style='zoom:50%'/>

#### 2. 非周期（aperiodic）

设有马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，其状态空间为$\mathcal{S}$，对任意状态$i\in \mathcal{S}$，如果时刻0从状态$i$出发，$t$时刻返回状态的所有时间长$\{t:\ P(X_{t}=i|X_{\color{red}{0}}=i)>0\}$的最大公约数是1，则称此马尔科夫链$X$是<font color=blue>非周期的（aperiodic）</font>，否则称$X$是周期的（periodic）。

直观上，<font color=blue>一个非周期的马尔科夫链，不存在一个状态，从这个状态出发，再返回该状态所经历的时长呈一定的周期性。</font> 

<img src='19.2_周期马尔科夫链.png' style='zoom:50%'/>

#### 定理19.2 
<font color='#c63c26'>不可约且非周期的**有限状态**马尔科夫链，有唯一平稳分布存在。</font> 

#### 3. 正常返（positive recurrent）

设有马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，其状态空间为$\mathcal{S}$，对任意状态$i,j \in \mathcal{S}$，定义概率$p_{i j}^{t}$（$p_{\color{red}{i \leftarrow j}}^{t}$）为时刻0从状态$j$ 出发，时刻$t$ 首次转移到状态$i$ 的概率，即$p_{i j}^{t}=P\left(X_{t}=i, X_{s} \neq i, s=1,2, \cdots, t-1 \mid X_{0}=j\right), t=1,2, \cdots$。若对所有状态$i,j$都满足$\lim _{t \rightarrow \infty} p_{i j}^{t}>0$，则称马尔科夫了$X$是正常返的（positive recurrent）。

直观上，<font color=blue>一个正常返的马尔科夫链，其中任意一个状态，从其他任意一个状态出发，当时间趋于无穷时，首次转移到这个状态的概率不为0。</font> 

<img src='19.2_正常返.png' style='zoom:50%'/>

#### 定理19.3 

<font color='#c63c26'>不可约、非周期且正常返的马尔科夫链，有唯一平稳分布存在。</font> 

#### 定理19.4（遍历定理）

设有马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，其状态空间为$\mathcal{S}$，若$X$是<font color=red>不可约、非周期且正常返的</font>，则该马尔科夫链有唯一平稳分布$\pi=\left(\pi_{1}, \pi_{2}, \cdots\right)^{\mathrm{T}}$，并且转移概率的极限分布是马尔科夫链的平稳分布  

$$
\lim_{\color{red}{t\rightarrow \infty}}P(X_{t}=i|X_{\color{red}{0}}=j)=\pi_{i},\quad i=1,2,\cdots\quad j=1,2,\cdots \tag{19.25}
$$ 

若$f(X)$是定义在状态空间$\mathcal{S}$上的函数，$E_{\pi}[|f(X)|]<\infty$，则  

$$
\begin{align}
& P\{ \hat{f}_{t} \rightarrow E_{\pi}[f(X)] \} = 1 \tag{19.26} \\
& \hat{f}_{t} = \frac{1}{t}\sum_{s=1}^{t}f(x_{s}) \\
& E_{\pi}[f(X)] = \sum_{i}f(i)\pi_{i}
\end{align}
$$

这里，$E_{\pi}[f(X)]$是$f(X)$关于平稳分布$\pi=\left(\pi_{1}, \pi_{2}, \cdots\right)^{\mathrm{T}}$的数学期望，（19.26）表示 

$$
\hat{f}_{t} \rightarrow E_{\pi}[f(X)], \quad t \rightarrow \infty \tag{19.27}
$$

几乎处处成立或以概率1成立。

**遍历定理的直观解释：**     
1. 满足相应条件的马尔科夫链，<font color=blue>当时间趋于无穷时，马尔科夫链的状态分布趋近于平稳分布，</font> 随机变量的函数的样本均值以概率1收敛于该函数的数学期望。


2. 样本均值可以认为是时间均值，而数学期望是空间均值。遍历定理实际上表述了便利性的含义: <font color=blue>当时间趋于无穷时，时间均值等于空间均值。</font>  


3. 遍历定理的三个条件：不可约、非周期、正常返，保证了当时间趋于无穷时达到任意一个状态的概率不为0.

理论上并不知道经过多少次迭代，马尔科夫链的状态分布才能接近于平稳分布，在实际应用遍历定理时，取一个足够大的整数$m$，经过$m$次迭代之后认为状态分布就是平稳分布，这时从$m+1$次迭代到第$n$次迭代的均值，即 

$$
\hat{E}f = \frac{1}{n-m}\sum_{i=m+1}^{n}f(x_{i}) \tag{19.28}
$$ 

称为<font color=blue>遍历均值。</font>  

**补充**  

$$
\pi(t) = P \pi(t-1) = P^{(t)} \pi(0)
$$

对于随机矩阵$P$，有特征值的绝对值$\leq 1$，$P$可对角化为  

$$
P = A \Lambda A^{\mathrm{-1}}
$$

$\Lambda = \mathrm{diag}(\lambda_{1}, \lambda_{2}, \cdots, \lambda_{K})$，有$\lambda_{i} \leq 1 \ \mathrm{for} \ i=1,2,\cdots,K$，不妨设只有$\lambda_{i}=1$，则 

$$
\pi(t) = (A\Lambda A^{\mathrm{-1}})^{t}\pi(0)=A \Lambda^{t}A^{\mathrm{-1}} \pi(0)
$$

存在足够大的$m$，使得
$$
\Lambda^{m} = \begin{bmatrix}
 0 &  &  &  & \\
  & \ddots  &  &  & \\
  &  & 1 &  & \\
  &  &  & \ddots & \\
  &  &  &  & 0
\end{bmatrix}
$$

因此有

$\pi(m+1) = A \Lambda^{m+1}A^{\mathrm{-1}} \pi(0) = A \Lambda^{m}A^{\mathrm{-1}} \pi(0)$  

$\pi(m+2) = A \Lambda^{m+2}A^{\mathrm{-1}} \pi(0) = A \Lambda^{m}A^{\mathrm{-1}} \pi(0)$  

所以，当$t>m$时，有 

$$
\pi(m+1) = \pi(m+2) = \cdots = \pi(m) = \cdots
$$

#### 5. 可逆马尔科夫链（reversible Markov chain）

设有马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，其状态空间为$\mathcal{S}$，转移概率矩阵为$P$，如果状态分布$\pi=\left(\pi_{1}, \pi_{2}, \cdots\right)^{\mathrm{T}}$，对于任意状态$i,j \in \mathcal{S}$，对于任意一个时刻$t$满足  

$$
P(X_{t}=i|X_{t-1}=j)\pi_{j} = P(X_{t-1}=j|X_{t}=i)\pi_{i},\quad i,j=1,2,\cdots \tag{19.29}
$$  

或简写为  

$$
\color{red}{p_{ij}\pi_{j} = p_{ji}\pi_{i}}, \quad i,j=1,2,\cdots \tag{19.30}
$$

则称此$X$为<font color=blue>可逆马尔科夫链</font>，式（19.30）称为<font color=red>细致平衡方程（detailed balance equation）</font>。

直观上，如果有可逆的马尔科夫链，那么以平稳分布作为初始分布，进行随机状态转移，<font color=blue>无论是面向未来还是过去，任何一个时刻的状态分布都是该平稳分布。</font> 

<img src='19.2_可逆马尔科夫链.png' style='zoom:50%'/>

In [10]:
import numpy as np
from numpy.linalg import eig

In [66]:
P = np.array([
    [1/2, 1/2, 1/4],
    [1/4, 0, 1/4],
    [1/4, 1/2, 1/2]
])

pi = np.array([
    [2/5, 1/5, 2/5]
])


for i in range(3):
    for j in range(3):
        print(P[i][j] * pi[0][j] == P[j][i] * pi[0][i], end=' ')

True True True True True True True True True 

In [67]:
P = np.array([
    [1/4, 1/2, 1/4],
    [1/4, 0, 1/2],
    [1/2, 1/2, 1/4]
])

pi = np.array([
    [8/25, 7/25, 2/5]
])

for i in range(3):
    for j in range(3):
        print(P[i][j] * pi[0][j] == P[j][i] * pi[0][i], end=' ')

True False False False True False False False True 

#### 定理19.5（细致平衡方程）
满足细致平衡方程的状态分布$\pi$就是该马尔科夫链的平稳分布，即  

$$
P\pi = \pi
$$

说明，<font color=blue>可逆马尔科夫链一定有唯一平稳分布，给出了一个马尔科夫链有平稳分布的充分条件（不是必要条件）。</font>也就是说，马尔科夫链满足遍历定理19.4的条件。

## 19.3 MCMC

### 19.3.1 基本思想

1. 相较传统的蒙特卡罗法（接受-拒绝法、重要性抽样法），MCMC更适合于<font color=blue>随机变量是多元的，p.d.f是非标准形式的，随机变量分量不是独立等情况。</font>  


2. **所要解决的问题**  
假设多元随机变量$x\in \mathcal{X}$，其概率密度函数为$P(x)$，$f(x)$是定义在$x\in \mathcal{X}$上的函数，目标是获得$p(x)$分布的样本，以及函数$f(x)$的数学期望$E_{p(x)}[f(x)]$.  


3. <font color=red>MCMC基本想法：</font>  
在随机变量$x$的状态空间$\mathcal{S}$上定义一个满足遍历定理的马尔科夫链$X=\left\{X_{0}, X_{1}, \cdots, X_{t}, \cdots\right\}$，使其平稳分布就是抽样的目标分布$p(x)$。然后在这个马尔科夫链上随机游走，每个时刻得到一个样本。<font color=blue>根据遍历定理，当时间趋于无穷时，样本的分布趋于平稳分布，样本函数均值趋近函数的数学期望。</font>所以，当时间足够长（时刻大于某个正整数$m$），在之后的时间（时刻小于等于某个正整数$n,\ n>m$）里随机游走得到样本集合$\left\{x_{m+1}, x_{m+2}, \cdots, x_{n}\right\}$就是目标概率分布的抽样结果，得到的函数均值就是要计算的数学期望:   
$$
\hat{E}f = \frac{1}{n-m}\sum_{i=m+1}^{n}f(x_{i}) \tag{19.32}
$$
到时刻$m$为止的时间段为<font color=red>燃烧期（burn-in, mixing time）</font>  

4. <font color=red>问题的关键：</font>  
如何构造具体的马尔科夫链。也就是<font color=red>如何定义转移核函数或转矩矩阵。</font>常用的方法有Metropolis-Hasting法、Gibbs算法。


5. 根据遍历定理可知: 随机游走的起始点并不影响结果，当时间趋于无穷，都会收敛到同一平稳分布。  


6. 收敛性判断（燃烧期$m$的取值）:  
没有具体理论可以计算，是经验性的。比如每隔一段时间取一次样本，得到多个样本后，计算遍历均值，当计算的均值稳定后，认为马尔科夫链已经收敛。  


7. 关于样本独立:  
MCMC抽取的样本，<font color=red>相邻样本点是相关的</font>，而不是独立的。可以在该样本序列中再次进行随机抽样，得到的样本子集合作为独立样本集合。

### 19.3.2 基本步骤

（1）首先，在随机变量$x$的状态空间$\mathcal{S}$构造一个满足遍历定理的马尔科夫链，使其平稳分布为目标分布$p(x)$；  


（2）从状态空间的某一点$x_{0}$出发，用构造的马尔科夫链进行随机游走，产生样本序列$x_{0}, x_{1}, \cdots, x_{t}, \cdots$


（3）应用马尔科夫链的遍历定理，确定正整数$m$和$n(m < n)$，得到样本集合$\left\{x_{m+1}, x_{m+2}, \cdots, x_{n}\right\}$，求得函数$f(x)$的均值（遍历均值）

$$
\hat{E}f = \frac{1}{n-m}\sum_{i=m+1}^{n}f(x_{i}) \tag{19.33}
$$

### 19.3.3 MCMC与统计学习

在贝叶斯学习中，MCMC可以用在模型的学习和推理上。

**贝叶斯学习框架:**  
观测数据由随机变量$y \in \mathcal{Y}$表示，模型由随机变量$x \in \mathcal{X}$表示，贝叶斯学习通过计算给定数据下模型的后验概率，并选择后验最大的模型:  

$$
\underset{\color{red}{后验}}{p(x|y)} = \frac{\overset{\color{red}{先验}}{p(x)} \overset{\color{red}{似然}}{p(y|x)} }{\int_{\mathcal{X}}p(y|x^{\prime})p(x^{\prime})\text{d}x^{\prime}} \tag{19.34}
$$

贝叶斯学习中经常需要进行三种积分运算: 规范化、边缘化、数学期望。  

规范化计算: 

$$
\int_{\mathcal{X}}p(y|x^{\prime})p(x^{\prime})\text{d}x^{\prime}
$$  

如有隐变量$z \in \mathcal{Z}$，后验的计算需要边缘化计算:  

$$p(x | y)=\int_{\mathcal{Z}} p(x, z | y) \mathrm{d} z$$  

期望计算:  

$$E_{P(x | y)}[f(x)]=\int_{\mathcal{X}} f(x) p(x | y) \mathrm{d} x$$  

当观测数据和模型都很复杂时，以上积分计算变得困难。MCMC为这些计算提供了一个通用有效的解决方案。